In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import itertools
from collections import defaultdict
from collections import Counter
import io
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
from itertools import groupby
from random import sample
import math
import json
import random
import os
import csv

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ['Computer Modern Roman'],
})
%matplotlib ipympl

In [1]:
jupyter server extension list

SyntaxError: invalid syntax (2045445874.py, line 1)

In [40]:
def generate_random_binary_vector(n):
    """
    Generates a random binary vector of length n.

    Args:
        n (int): The length of the vector.

    Returns:
        numpy.ndarray: A binary vector of length n.
    """
    return np.random.randint(2, size=n)

def generate_noise_vector(m,c):
    return np.random.randint(2*c+1, size=m) - np.array([c]*m)
    
def generate_powerset_matrix(n, m, randomise = True):
    """
    Generates a binary matrix of size m-by-n where the rows generate every set in the powerset of a length n vector.

    Args:
        n (int): The length of the input vector.
        m (int): The number of rows in the output matrix.

    Returns:
        numpy.ndarray: A binary matrix of size m-by-n.
    """
    if randomise == False:
        powerset = []
        for i in range(min(m,2**n)):
            subset = [int(b) for b in bin(i)[2:].zfill(n)]
            powerset.append(np.array(subset))
            
    return np.array(powerset)
    
    # Generate the powerset of the input vector
    powerset = list(itertools.product([0, 1], repeat=n))
    
    if randomise == True:
        random.shuffle(powerset)

    # Ensure that the powerset is sorted by number of ones
    # powerset.sort(key=lambda x: sum(x))

    # Create the matrix by selecting the first m elements of the powerset
    matrix = np.array(powerset[:m])

    return matrix

# define a dictionary to map integer status values to string status values
status_dict = {
    gp.GRB.OPTIMAL: "OPTIMAL",
    gp.GRB.INFEASIBLE: "INFEASIBLE",
    gp.GRB.INF_OR_UNBD: "INF_OR_UNBD",
    gp.GRB.UNBOUNDED: "UNBOUNDED",
    gp.GRB.CUTOFF: "CUTOFF",
    gp.GRB.ITERATION_LIMIT: "ITERATION_LIMIT",
    gp.GRB.NODE_LIMIT: "NODE_LIMIT",
    gp.GRB.TIME_LIMIT: "TIME_LIMIT",
    gp.GRB.SOLUTION_LIMIT: "SOLUTION_LIMIT",
    gp.GRB.INTERRUPTED: "INTERRUPTED",
    gp.GRB.SUBOPTIMAL: "SUBOPTIMAL",
    gp.GRB.NUMERIC: "NUMERIC",
}



def reconstruct_LP_OF(n,c,A,y):
    M = gp.Model()
    M.Params.OutputFlag = 0
    x = M.addMVar(n, lb=0, ub=1, vtype=gp.GRB.CONTINUOUS, name='x')
    e = M.addMVar(len(A), lb=-c, ub=c, vtype=gp.GRB.CONTINUOUS, name='e')
    
    #need these to linearise the L1 norm..
    e_lin = M.addMVar(len(A), vtype=gp.GRB.CONTINUOUS, name='e_lin')
    # t = M.addVar(vtype=gp.GRB.CONTINUOUS, name='t')
    
    
    for i in range(len(A)):
        M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) + e[i] == y[i])
        # M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) >= y[i] - c)
        
        M.addConstr(e[i] <= e_lin[i])
        M.addConstr(-e_lin[i] <= e[i])
        
        
    # M.addConstr(gp.quicksum(e[i] for i in range(len(A))) == t) 
    
    # M.setObjective(t, GRB.MINIMIZE)
    # M.setObjective(gp.quicksum(gp.abs_(i) for i in e), GRB.MINIMIZE)
    M.setObjective(gp.quicksum(e_lin), GRB.MINIMIZE)
    
    M.optimize()
    
    # retrieve the status as an integer
    status_int = M.status

    # convert the integer status to a string
    status_str = status_dict.get(status_int, "UNKNOWN")

    # print the status string
    # print("Status: " + status_str)
    
    if M.status == gp.GRB.OPTIMAL:
        reconstruction = x.X
        return np.array([round(x) for x in reconstruction])
    else:
        return None
    
    # reconstruction = x.X
    # return np.array([round(x) for x in reconstruction])

def reconstruct_LP(n,c,A,y):
    M = gp.Model()
    M.Params.OutputFlag = 0
    x = M.addMVar(n, lb=0, ub=1, vtype=gp.GRB.CONTINUOUS, name='x')
    
    for i in range(len(A)):
        M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) <= y[i] + c)
        M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) >= y[i] - c)

    M.optimize()
    
    reconstruction = x.X
    return np.array([round(x) for x in reconstruction])
    
def reconstruct_ILP(n,c,A,y):
    M = gp.Model()
    M.Params.OutputFlag = 0
    x = M.addMVar(n, lb=0, ub=1, vtype=gp.GRB.INTEGER, name='x')
    
    for i in range(len(A)):
        M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) <= y[i] + c)
        M.addConstr(gp.quicksum(A[i,j]*x[j] for j in range(n)) >= y[i] - c)

    M.optimize()
    
    # Parameters
    M.Params.PoolSearchMode = 2
    M.Params.PoolSolutions = 1000
    # m.Params.PoolSolutions = 2
    M.Params.PoolGap = 0.0

    # Optimize
    M.optimize()
    
    # print(f"Took {M.Runtime:.2f} seconds to solve")
    # print(f"Found {M.SolCount} solutions")
    
    return(M.SolCount)
    
    temp = []
    for k in range(M.SolCount):
        M.Params.SolutionNumber = k
        # temp.append([x.Xn for x in quers_vars.values()])
        # temp.append([round(quers_vars[tuple(x)].Xn) for x in A])
        temp.append([round(var.Xn) for var in M.getVars()])
        
    return temp
    
    # if M.status == gp.GRB.OPTIMAL:
    #     reconstruction = x.X
    #     return np.array([round(x) for x in reconstruction])
    # else:
    #     return None
    
def binary_vector_pairs(v):
    """ generates all pairs of vectors which add to vector v"""
    n = len(v)
    pairs = []
    for i in range(2**n):
        b = bin(i)[2:].zfill(n)
        x = [int(j) for j in b]
        y = [(v[j] - x[j]) % 2 for j in range(n)]
        if np.array_equal(np.add(x,y), np.array(v)) and x <= y:
            pairs.append((x, y))
    return pairs

def gen_mc_noise(A,c,x = [], N_sols = 1000000):
    """
    Adds (bounded by c) noise to each of the queries in A, ensuring additivity
    is respected (margin consistent. 
    
    returns N_sols noise vectors corresponding to the query matrix A. 
    
    if x (database) is provided, it ensures non-negativity of resulting noisy queries. 
    
    """
    m = len(A)
    n = len(A[0])
    all_quers = list(itertools.product([0, 1], repeat=n))

    M = gp.Model()
    M.Params.OutputFlag = 0
    
    quers_vars = dict()
    for quer in all_quers:
        quers_vars[(quer)] = M.addVar(lb = -c, ub = c, vtype='I', name=f"{quer}")
        pairs = binary_vector_pairs(quer)
        for i,j in pairs:
            M.addConstr(quers_vars[tuple(i)] + quers_vars[tuple(j)] == quers_vars[tuple(quer)])
        
    if len(x)>0:
        for quer in all_quers:
            M.addConstr(np.matmul(quer,x) + quers_vars[(quer)] >= 0)
    
    
    
    
    # Parameters
    M.Params.PoolSearchMode = 2
    M.Params.PoolSolutions = N_sols
    # m.Params.PoolSolutions = 2
    M.Params.PoolGap = 0.0

    # Optimize
    M.optimize()
    
    # print(f"Took {M.Runtime:.2f} seconds to solve")
    print(f"Found {M.SolCount} solutions")
    
    temp = []
    for k in range(M.SolCount):
        M.Params.SolutionNumber = k
        # temp.append([x.Xn for x in quers_vars.values()])
        temp.append([round(quers_vars[tuple(x)].Xn) for x in A])
        # temp.append([round(var.Xn) for var in M.getVars()])
        
    return temp

In [47]:
n = 5
m = 2**n
c = 1
N_sols = 2

x = generate_random_binary_vector(n)
A = generate_powerset_matrix(n, m, randomise = False)
# print(A)
b = generate_noise_vector(len(A),c)

print(x)
# print(A)
# print(b)
# print(np.matmul(A,x))

# y = np.matmul(A,x)+b
# print('random',reconstruct_ILP(n,c,A,y))


# poss_bs = gen_mc_noise(A,c,x,N_sols)
poss_bs = [generate_noise_vector(len(A),c) for i in range(N_sols)]


for j in poss_bs:
    y = np.matmul(A,x)+j
    print(reconstruct_LP(n,c,A,y))
    print(reconstruct_LP_OF(n,c,A,y))
    print(reconstruct_ILP(n,c,A,y))
    
# numbers[(n,c)] = gen_mc_noise(A,c)
# print(numbers)

[1 1 0 0 0]
[1 1 0 0 0]
[1 1 0 0 0]
1
[1 1 0 0 0]
[1 1 0 0 0]
1


In [19]:
import random

def generate_powerset(n, m):
    powerset = []
    count = 0
    while count < m:
        subset = [0] * n
        for i in random.sample(range(n), random.randint(0, n)):
            subset[i] = 1
        if subset not in powerset:
            powerset.append(subset)
            count += 1
    return powerset

powerset = generate_powerset(4, 5)
print(powerset)


[[0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 1], [1, 1, 0, 0], [0, 1, 1, 0]]


In [37]:
def generate_powerset(n, m):
    powerset = []
    for i in range(m):
        subset = [int(b) for b in bin(i)[2:].zfill(n)]
        powerset.append(subset)
    return powerset

print(generate_powerset(3,100))

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 0], [1, 0, 1, 1, 1, 1], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1], [1, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 1], [1, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 1, 0], [1, 1, 0, 1, 1, 1], [1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 1], [1, 1, 